In [78]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import os
from google.colab import drive
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, Bidirectional, LSTM
from keras.layers import Activation, Dense

In [79]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [80]:
datafiles_path = "/content/drive/My Drive/ML_Assignment3/data"
datafiles_names = os.listdir(path=datafiles_path)

In [81]:
print(datafiles_names[:5])

['da0c1f6ca5343f82', '6e7380081c3005fb', '08d560d3d168cebc', 'b15b2cf0065a50ca', '5d28a06904606d82']


In [82]:
print(len(datafiles_names))

1149


In [83]:
corpus_array = []

for i in range(len(datafiles_names)):
    path = os.path.join(datafiles_path,datafiles_names[i])
    try:
      if 'RESULTS' in open(path,'r').read():
         corpus_array.append(open(path).read()[open(path,'r').read().find('== RESULTS') + 10:open(path,'r').read().find('== ISSUES')].replace('\n',''))
    except:
        pass

In [84]:
corpus_array = corpus_array[:50]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus_array)
all_words_count = len(tokenizer.word_index) + 1

In [85]:
print(all_words_count)

1047


In [86]:
input_seq = []
for line in corpus_array:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_seq = token_list[:i + 1]
        input_seq.append(n_gram_seq)

In [87]:
max_seq_len = max([len(x) for x in input_seq])

In [88]:
input_seq = np.array(pad_sequences(input_seq,
 maxlen=max_seq_len, padding='pre'))

In [89]:
xs = input_seq[:, :-1]
labels = input_seq[:, -1]

In [90]:
xs = xs.astype(np.int16)
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words_count)

In [91]:
model = tf.keras.models.Sequential()
model.add(Embedding(input_dim=total_words_count, output_dim=250, input_length=max_sequence_len - 1))
model.add(Bidirectional(LSTM(10, return_sequences=True), input_shape=(5, 10)))
model.add(Bidirectional(LSTM(10)))
model.add(Dense(total_words_count))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [92]:
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 245, 250)          261750    
                                                                 
 bidirectional_6 (Bidirectio  (None, 245, 20)          20880     
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 20)               2480      
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 1047)              21987     
                                                                 
 activation_3 (Activation)   (None, 1047)              0         
                                                                 
Total params: 307,097
Trainable params: 307,097
Non-tr

In [93]:
hist = model.fit(xs, ys, epochs=20, verbose=1)

Epoch 1/20
78/78 [==============================] - 31s 317ms/step - loss: 6.7481 - accuracy: 0.0384
Epoch 2/20
78/78 [==============================] - 25s 314ms/step - loss: 6.2842 - accuracy: 0.0514
Epoch 3/20
78/78 [==============================] - 24s 312ms/step - loss: 6.1650 - accuracy: 0.0514
Epoch 4/20
78/78 [==============================] - 25s 315ms/step - loss: 6.1190 - accuracy: 0.0514
Epoch 5/20
78/78 [==============================] - 25s 322ms/step - loss: 6.0564 - accuracy: 0.0514
Epoch 6/20
78/78 [==============================] - 25s 315ms/step - loss: 5.9970 - accuracy: 0.0514
Epoch 7/20
78/78 [==============================] - 25s 321ms/step - loss: 5.9470 - accuracy: 0.0534
Epoch 8/20
78/78 [==============================] - 25s 314ms/step - loss: 5.9118 - accuracy: 0.0554
Epoch 9/20
78/78 [==============================] - 25s 318ms/step - loss: 5.8839 - accuracy: 0.0526
Epoch 10/20
78/78 [==============================] - 25s 316ms/step - loss: 5.8444 - accura

In [95]:
model.save('unsupervised_model1')

INFO:tensorflow:Assets written to: unsupervised_model1/assets


INFO:tensorflow:Assets written to: unsupervised_model1/assets


In [96]:
model1 = tf.keras.models.load_model('unsupervised_model1')

In [105]:
def generate_text(seed_text,next_words,model):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
        predict_x=model1.predict(token_list)
        classes_x=np.argmax(predict_x,axis=1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == classes_x:
                output_word = word
                break
        seed_text += ' ' + output_word
    return seed_text
generate_text('AI',10,model1)

'AI the model the results the results the results of of'